# Starting with just 2023 for my laptop 

In [2]:
import duckdb
import pandas as pd
import pandas_ta

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")
print(con)

In [22]:
tables = con.execute('PRAGMA show_tables;').fetchall()
print(tables)

[('Stocks',)]


In [31]:
qry = '''
    SELECT * FROM Stocks
    WHERE EXTRACT(YEAR FROM Datetime) = 2023 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry).fetchdf()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220495 entries, 0 to 220494
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  220495 non-null  datetime64[us]
 1   open      220495 non-null  float32       
 2   high      220495 non-null  float32       
 3   low       220495 non-null  float32       
 4   close     220495 non-null  float32       
 5   volume    220495 non-null  float32       
 6   Stock     220495 non-null  object        
 7   interval  220495 non-null  float32       
dtypes: datetime64[us](1), float32(6), object(1)
memory usage: 8.4+ MB
None


# Creating and Flattening the Data for:
#### Packet Size (Only 1 min intervals)

### Getting the Data, and calculating P/L

In [ ]:
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time']>start_time)&(df['Time']<end_time)]

def trail_stoploss(index, tradingdata, whole_percentage):
    idx = index
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 20 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] # Get the dataframe from the row you want until 100 minutes later
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
   
    for index, row in after_td.iterrows():
        if time_counter <= time_stop:
            close = row['close']
            if close > max_price:
                max_price = close
            elif close < min_price:
                profit = close - basis
                return [index, profit]
            else:
                pass
        else:
            pass
            profit = close - basis
            return [index, profit]
            
    




In [48]:
for index, row in TradingDay_df.iterrows():
    packet100 = df.loc[index-99:index].info() # Open, Close, EMA 20 , EMA 100, RSI 
    packet50 = df.loc[index-49:index].info() # Open, Close, EMA 20, EMA 50, RSI 
    packet20 = df.loc[index-19:index].info() # Open, Close, EMA 8, EMA 20, RSI
    packet10 = df.loc[index-9:index].info() # Open, Close, EMA 10, RSI

    packet100['EMA100'] = None
    packet100['EMA20'] = None
    packet100['RSI'] = None

    packet100['EMA50'] = None
    packet100['EMA20'] = None
    packet100['RSI'] = None

    packet100['EMA20'] = None
    packet100['EMA8'] = None
    packet100['RSI'] = None

    packet100['EMA10'] = None
    packet100['RSI'] = None
   


143.32000732421875
